<a href="https://cognitiveclass.ai"><img src = "https://ibm.box.com/shared/static/9gegpsmnsoo25ikkbl4qzlvlyjbgxs5x.png" width = 400> </a>

<h1 align=center><font size = 5>Segmenting and Clustering Neighborhoods in Toronto</font></h1>


## Notebook for Applied Data Science Capstone - Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto by Emil Fuerstenberg Haegg

### Import and install necessary packages

In [3]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analysis

# Set diplay options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import requests # library to handle requests

#!pip install bs4  # uncomment if not already installed
from bs4 import BeautifulSoup

print('Libraries imported.')

Libraries imported.


### Scrape data from HTML tables into a DataFrame using BeautifulSoup and Pandas

In [4]:
# Webpage URL
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

In [5]:
# Getting content in text format and store in variable 'data'
data  = requests.get(url).text

In [6]:
# Creating beautifulsoup object on 'data'
soup = BeautifulSoup(data,"html.parser")

In [7]:
# Find all html-tables in the webpage
tables = soup.find_all('table') # in html table is represented by the tag <table>

In [8]:
# Check how many tables were found
len(tables)

3

In [9]:
# Indices for tables found  
for index,table in enumerate(tables):
    table_index = index
    print(table_index)

0
1
2


In [10]:
# Check for name of tables
for index, table in enumerate(tables):
    print(tables[index].caption)

None
None
None


In [11]:
# Inspecting webpage and finding that table in interest is wikitable, 
# testing find all for that class, rather than looking through tables above
wikitables = soup.find_all("table",{"class":"wikitable"})
# number of tables
len(wikitables)

1

In [12]:
# Table in interest
tablen = wikitables[0]

In [13]:
# Create dataframe
postal_data = pd.DataFrame(columns=["PostalCode", "Borough", "Neighborhood"])

# Enumerating over tablerows and columns, fill dataframe with data from table
for row in tablen.tbody.find_all("tr"):
    col = row.find_all("td")
    if (col != []):
        PostalCode = col[0].text
        Borough = col[1].text
        Neighborhood = col[2].text.strip()
        postal_data = postal_data.append({"PostalCode":PostalCode, "Borough":Borough, "Neighborhood":Neighborhood}, ignore_index=True)

postal_data.head()

,PostalCode,Borough,Neighborhood
0,M1A\n,Not assigned\n,Not assigned
1,M2A\n,Not assigned\n,Not assigned
2,M3A\n,North York\n,Parkwoods
3,M4A\n,North York\n,Victoria Village
4,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront"


### Process dataframe according to instructions

In [14]:
# Remove(replace with empty string) newline character 
postal_data = postal_data.replace('\n', '', regex=True)

postal_data.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [15]:
# Create varible with True if condition met
selection = postal_data["Borough"] != "Not assigned"

# Selection from table using varieble
postal_data = postal_data[selection]

# If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough
postal_data.loc[postal_data.Neighborhood == "Not assigned", 'Neighborhood'] = postal_data["Borough"]

# Reset index
postal_data.reset_index(drop=True, inplace=True)

postal_data.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [16]:
# Number of rows and columns in dataframe
postal_data.shape

(103, 3)